Simple folder and its files view , not tree view

In [1]:
from googleapiclient.discovery import build
from google.colab import auth
from google.auth import default
from google.colab import drive
import os

# ✅ Drive Mount
drive.mount('/content/drive')

# ✅ Authentication
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

# ✅ Output File Path
output_file = "/content/drive/MyDrive/drive_full_view_sorted.txt"

# ✅ All Files Fetch Function
def list_drive_files():
    results = []
    page_token = None

    while True:
        response = service.files().list(
            q="trashed=false",
            spaces="drive",
            fields="nextPageToken, files(id, name, mimeType, parents)",
            pageSize=1000,
            pageToken=page_token
        ).execute()

        results.extend(response.get('files', []))
        page_token = response.get('nextPageToken')

        if not page_token:
            break

    return results


files = list_drive_files()

# ✅ Separate folders and files
folders = [f for f in files if f["mimeType"] == "application/vnd.google-apps.folder"]
normal_files = [f for f in files if f["mimeType"] != "application/vnd.google-apps.folder"]

# ✅ Sort folders A–Z
folders.sort(key=lambda x: x["name"].lower())

# ✅ Writing Sorted Output
with open(output_file, "w", encoding="utf-8") as f:

    for folder in folders:
        f.write(f"\n📁 FOLDER: {folder['name']}\n")

        # ✅ Get files inside this folder
        folder_files = [
            file for file in normal_files
            if "parents" in file and folder["id"] in file["parents"]
        ]

        # ✅ Sort files inside folder A–Z
        folder_files.sort(key=lambda x: x["name"].lower())

        if folder_files:
            for file in folder_files:
                f.write(f"    📄 {file['name']}\n")
        else:
            f.write("    (No files)\n")

print("✅ A–Z Sorted Google Drive View TXT File Ready:")
print(output_file)


Mounted at /content/drive


✅ A–Z Sorted Google Drive View TXT File Ready:
/content/drive/MyDrive/drive_full_view_sorted.txt
